In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

import warnings

import surprise
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import train_test_split

warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('맥주_10_732개.csv', encoding='utf-8', index_col=0)

In [3]:
data = pd.read_csv('맥주_5.csv', encoding='utf-8', index_col=0)

In [4]:
ratings = data.copy()

In [5]:
data

,아이디,맥주,평점
1,"pictoman(1,959)",Kloud Original Gravity,2.0
2,"snoworsummer(8,581)",Kloud Original Gravity,3.0
4,"GuideDogSaint(2,549)",Kloud Original Gravity,2.2
5,"deanso(13,380)",Kloud Original Gravity,2.3
7,"DrNosha(5,865)",Kloud Original Gravity,2.3
...,...,...,...
3,"TedE(2,324)",Queen's Ale Blonde Type,3.4
4,"KansaiBeerLvrs(7,283)",Queen's Ale Blonde Type,3.1
5,"cubs(10,772)",Queen's Ale Blonde Type,3.0
6,"arminjewell(9,756)",Queen's Ale Blonde Type,2.9


# 입력받은 데이터 병합

In [184]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234325 entries, 2 to 0
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   아이디         234325 non-null  object 
 1   맥주          234325 non-null  object 
 2   날짜          234325 non-null  object 
 3   평점          234325 non-null  float64
 4   편의점         61009 non-null   object 
 5   Aroma       234325 non-null  int64  
 6   Appearance  234325 non-null  int64  
 7   Flavor      234325 non-null  int64  
 8   Mouthfeel   234325 non-null  int64  
 9   Overall     234325 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 19.7+ MB


In [185]:
ratings['맥주'].unique()

array(['Kloud Original Gravity', 'Kloud Fitz Super Clear',
       'Asahi Super Dry', 'Tsingtao', 'Heineken', 'Kirin Ichiban',
       'Sapporo Premium Beer / Draft Beer', 'Stella Artois',
       'Guinness Draught', 'Kronenbourg 1664 Blanc', 'Pilsner Urquell',
       'San Miguel', 'OB Premier Pilsner', 'Cass Fresh',
       'Hite D (Dry Finish)', 'Hite Prime Max', 'Hite Extra Cold',
       'Victoria Bitter', 'Bintang Pilsener', 'Krombacher Weizen',
       'Miller Genuine Draft (MGD)', 'Tiger Radler Lemon',
       "Suntory The Premium Malt's", 'Reeper B. Blondes Weissbier',
       'Reeper B. Exotisches IPA', 'Tiger Beer', 'Tsingtao Wheat Beer',
       'Erdinger Weissbier', 'Carlsberg Pilsner', 'Budweiser',
       'Hoegaarden', 'Sapporo Yebisu', 'Paulaner Hefe-Weissbier',
       'Desperados', 'Peroni Nastro Azzurro',
       'Edelweiss Weissbier Snowfresh', 'Heineken Dark Lager',
       'Kozel Černý (Dark) 10°', 'Guinness Original 4.2% (Ireland/UK)',
       'Jeju Wit Ale', 'Stephans Bräu Pre

In [186]:
add1 = ['신구현', 'Kozel Černý (Dark) 10', 3.5]
add2 = ['신구현', 'Kronenbourg 1664 Blanc', 3.0]
add3 = ['신구현', 'Leffe Brune / Bruin / Brown', 4.0]
add4 = ['신구현', 'Tsingtao Wheat Beer', 4.0]
add5 = ['신구현', 'Hoegaarden', 4.0]

add1 = pd.DataFrame(data=[add1], columns=['아이디','맥주','평점'])
add2 = pd.DataFrame(data=[add2], columns=['아이디','맥주','평점'])
add3 = pd.DataFrame(data=[add3], columns=['아이디','맥주','평점'])
add4 = pd.DataFrame(data=[add4], columns=['아이디','맥주','평점'])
add5 = pd.DataFrame(data=[add5], columns=['아이디','맥주','평점'])

In [187]:
ratings = data.copy()

In [188]:
ratings = pd.concat([ratings, add1])
ratings = pd.concat([ratings, add2])
ratings = pd.concat([ratings, add3])
ratings = pd.concat([ratings, add4])
ratings = pd.concat([ratings, add5])

In [189]:
# 이름 설정
uname = '신구현'

In [190]:
ratings

,아이디,맥주,날짜,평점,편의점,Aroma,Appearance,Flavor,Mouthfeel,Overall
2,"snoworsummer(8,581)",Kloud Original Gravity,"March 7, 2020",3.0,CU GS25 7/11 ministop,6.0,3.0,6.0,3.0,12.0
3,mickmac(726),Kloud Original Gravity,"February 20, 2020",2.9,CU GS25 7/11 ministop,5.0,4.0,4.0,4.0,12.0
4,"GuideDogSaint(2,549)",Kloud Original Gravity,"February 8, 2020",2.2,CU GS25 7/11 ministop,4.0,2.0,5.0,2.0,9.0
5,"deanso(13,380)",Kloud Original Gravity,"January 16, 2020",2.3,CU GS25 7/11 ministop,4.0,3.0,4.0,3.0,9.0
7,"DrNosha(5,865)",Kloud Original Gravity,"December 20, 2019",2.3,CU GS25 7/11 ministop,4.0,3.0,4.0,3.0,9.0
...,...,...,...,...,...,...,...,...,...,...
0,신구현,Kozel Černý (Dark) 10,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN
0,신구현,Kronenbourg 1664 Blanc,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
0,신구현,Leffe Brune / Bruin / Brown,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
0,신구현,Tsingtao Wheat Beer,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN


# 피벗 테이블 변환

In [191]:
# 우리가 예측한 평점과 실제 평점간의 차이를 MSE로 계산
def get_mse(pred, actual):
    # 평점이 있는 실제 영화만 추출
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [192]:
# 특정 맥주와 비슷한 유사도를 가지는 맥주 Top_N에 대해서만 적용 -> 시간오래걸림

def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)
    
    # 사용자-아이템 평점 행렬의 맥주 개수만큼 루프
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개의 데이터 행렬의 인덱스 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점 계산 : 각 col 맥주별(1개), 2496 사용자들의 예측평점
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col,:][top_n_items].dot(
            ratings_arr[row, :][top_n_items].T)
            pred[row, col] /= np.sum(item_sim_arr[col,:][top_n_items])
            
    return pred

In [193]:
# 사용자가 안 먹어본 맥주를 추천하자.
def get_not_tried_beer(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 맥주 정보를 추출해 Series로 반환
    # 반환된 user_rating은 영화명(title)을 인덱스로 가지는 Series 객체
    user_rating = ratings_matrix.loc[userId, :]
    
    # user_rating이 0보다 크면 기존에 관란함 영화.
    # 대상 인덱스를 추출해 list 객체로 만듦
    tried = user_rating[user_rating>0].index.tolist()
    
    # 모든 맥주명을 list 객체로 만듦
    beer_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 tried에 해당하는 영화는 beer_list에서 제외
    not_tried = [beer for beer in beer_list if beer not in tried]
    
    return not_tried

In [194]:
# 예측 평점 DataFrame에서 사용자 id 인덱스와 not_tried로 들어온 맥주명 추출 후
# 가장 예측 평점이 높은 순으로 정렬
def recomm_beer_by_userid(pred_df, userId, not_tried, top_n):
    recomm_beer = pred_df.loc[userId, not_tried].sort_values(ascending=False)[:top_n]
    return recomm_beer

In [195]:
# 피벗 테이블로 변환 후 Nan값을 0으로 대체
ratings_matrix = ratings.pivot_table('평점', index='아이디', columns='맥주')
ratings_matrix = ratings_matrix.fillna(0)

# 트랜스포즈 : 아이템 - 유저로 변환
ratings_matrix_T = ratings_matrix.transpose()
# 아이템 - 아이템 계산
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)
# 데이터 프레임으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)

# top_n과 비슷한 유저들만 추천에 사용
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=5)
print('아이템 기반 최근접 TOP-N 이웃 MSE: ', 
      get_mse(ratings_pred, ratings_matrix.values))

# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                  columns=ratings_matrix.columns)

# 유저가 먹지 않은 맥주이름 추출
not_tried = get_not_tried_beer(ratings_matrix, uname)

# 아이템 기반의 최근접 이웃 CF로 맥주 추천
recomm_beer = recomm_beer_by_userid(ratings_pred_matrix, uname, not_tried, top_n=3)
recomm_beer = pd.DataFrame(data=recomm_beer.values, index=recomm_beer.index,
                          columns=['예측평점'])

recomm_beer

아이템 기반 최근접 TOP-N 이웃 MSE:  0.6340684862501499


,예측평점
맥주,
Edelweiss Weissbier Snowfresh,1.344598
Pilsner Urquell,1.163448
Paulaner Hefe-Weissbier,1.142857


In [18]:
recomm_beer.index.tolist()

['Edelweiss Weissbier Snowfresh',
 'Pilsner Urquell',
 'Paulaner Hefe-Weissbier',
 'Guinness Draught',
 'Franziskaner Hefe-Weissbier / Weissbier Naturtrub']

# MF를 사용한 추천시스템

In [53]:
def get_rmse(R, P, Q, non_zeros):
    error = 0
    
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스를 추출해 실제 R 행렬과 예측 행렬 RMSE
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [54]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape
    
    # P와 Q의 매트릭스 크기를 지정하고 정규 분포를 가진 랜덤한 값으로 입력
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))
    
    prev_rmse = 10000
    break_count = 0
    
    prev_rmse = 10000
    break_count = 0
    
    # R>0인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장.
    non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]
    
    # SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i,:], Q[j,:].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
            
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0:
            print('### iteration step : ', step, " rmse: ", rmse)
        
    return P,Q

In [55]:
import pandas as pd
import numpy as np

data = pd.read_csv('맥주_10.csv', encoding='utf-8', index_col=0)

In [56]:
ratings = data.copy()
ratings_matrix = ratings.pivot_table('평점', index='아이디', columns='맥주')
ratings_matrix

맥주,Asahi Super Dry,Asahi Super Dry Black,Bavaria 8.6 (Original),Bavaria Pilsener / Premium Beer,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,Budweiser,Carlsberg Pilsner,Cass Fresh,...,Stephans Bräu Premium Lager,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Victoria Bitter,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
아이디,,,,,,,,,,,,,,,,,,,,,
007lund(92),NaN,NaN,NaN,NaN,2.6,NaN,NaN,2.0,1.8,NaN,...,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,4.1
"00cobraR(1,103)",NaN,NaN,NaN,NaN,2.3,NaN,NaN,1.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.1,NaN,NaN,NaN,NaN
1010(309),2.4,NaN,NaN,NaN,2.0,NaN,NaN,0.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12HundredBaud(262),2.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16erBlech(561),NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,2.0,NaN,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zenkis(954),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ziobarba2000(91),NaN,NaN,NaN,1.6,1.6,NaN,NaN,2.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.4
zombywoof(483),NaN,NaN,NaN,1.5,1.8,NaN,NaN,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5


In [57]:
ratings_matrix.values

array([[nan, nan, nan, ..., nan, nan, 4.1],
       [nan, nan, nan, ..., nan, nan, nan],
       [2.4, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, 2.5],
       [1.2, 2.5, nan, ..., 1.4, nan, 2.3],
       [nan, nan, nan, ..., nan, nan, 2.5]])

In [58]:
P,Q = matrix_factorization(ratings_matrix.values, K=10, steps=200, learning_rate=0.01, r_lambda=0.02)

pred_matrix = np.dot(P, Q.T)

### iteration step :  0  rmse:  1.595830433880674
### iteration step :  10  rmse:  0.4837890913524185
### iteration step :  20  rmse:  0.44526393274784387
### iteration step :  30  rmse:  0.40955763531456063
### iteration step :  40  rmse:  0.37861149059058763
### iteration step :  50  rmse:  0.35864720379910187
### iteration step :  60  rmse:  0.34596832325234006
### iteration step :  70  rmse:  0.3373986407488805
### iteration step :  80  rmse:  0.33123848514319804
### iteration step :  90  rmse:  0.32659799685978974
### iteration step :  100  rmse:  0.3229790119239775
### iteration step :  110  rmse:  0.3200805105997842
### iteration step :  120  rmse:  0.31770927514086617
### iteration step :  130  rmse:  0.31573541294066104
### iteration step :  140  rmse:  0.31406828172379353
### iteration step :  150  rmse:  0.31264262857423947
### iteration step :  160  rmse:  0.3114102349121014
### iteration step :  170  rmse:  0.31033468226310623
### iteration step :  180  rmse:  0.3093879564

## 테스트 : 학습된 P,Q를 Input으로 주고 학습하자. 

In [135]:
ratings = pd.read_csv('맥주_10.csv', encoding='utf-8', index_col=0)

In [136]:
# 피벗 테이블로 변환 후 Nan값을 0으로 대체
ratings_matrix = ratings.pivot_table('평점', index='아이디', columns='맥주')
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix

맥주,Asahi Super Dry,Asahi Super Dry Black,Bavaria 8.6 (Original),Bavaria Pilsener / Premium Beer,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,Budweiser,Carlsberg Pilsner,Cass Fresh,...,Stephans Bräu Premium Lager,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Victoria Bitter,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
아이디,,,,,,,,,,,,,,,,,,,,,
007lund(92),0.0,0.0,0.0,0.0,2.6,0.0,0.0,2.0,1.8,0.0,...,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,4.1
"00cobraR(1,103)",0.0,0.0,0.0,0.0,2.3,0.0,0.0,1.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0
1010(309),2.4,0.0,0.0,0.0,2.0,0.0,0.0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12HundredBaud(262),2.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16erBlech(561),0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zenkis(954),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ziobarba2000(91),0.0,0.0,0.0,1.6,1.6,0.0,0.0,2.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4
zombywoof(483),0.0,0.0,0.0,1.5,1.8,0.0,0.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5


In [138]:
ratings.맥주.unique()

array(['Kloud Original Gravity', 'Kloud Fitz Super Clear',
       'Asahi Super Dry', 'Tsingtao', 'Heineken', 'Kirin Ichiban',
       'Sapporo Premium Beer / Draft Beer', 'Stella Artois',
       'Guinness Draught', 'Kronenbourg 1664 Blanc', 'Pilsner Urquell',
       'San Miguel', 'OB Premier Pilsner', 'Cass Fresh',
       'Hite D (Dry Finish)', 'Hite Prime Max', 'Hite Extra Cold',
       'Victoria Bitter', 'Bintang Pilsener', 'Krombacher Weizen',
       'Miller Genuine Draft (MGD)', 'Tiger Radler Lemon',
       "Suntory The Premium Malt's", 'Reeper B. Exotisches IPA',
       'Tiger Beer', 'Tsingtao Wheat Beer', 'Erdinger Weissbier',
       'Carlsberg Pilsner', 'Budweiser', 'Hoegaarden', 'Sapporo Yebisu',
       'Paulaner Hefe-Weissbier', 'Desperados', 'Peroni Nastro Azzurro',
       'Edelweiss Weissbier Snowfresh', 'Heineken Dark Lager',
       'Kozel Černý (Dark) 10°', 'Guinness Original 4.2% (Ireland/UK)',
       'Jeju Wit Ale', 'Stephans Bräu Premium Pilsener',
       'Stephans Bräu 

In [139]:
add1 = ['신구현', 'Asahi Super Dry', 3.5]
add2 = ['신구현', 'Hite Prime Max', 3.0]
add3 = ['신구현', 'Holsten Premium Bier', 4.0]
add4 = ['신구현', 'Erdinger Weissbier Dunkel', 4.0]
add5 = ['신구현', 'Red Rock', 4.0]

add1 = pd.DataFrame(data=[add1], columns=['아이디','맥주','평점'])
add2 = pd.DataFrame(data=[add2], columns=['아이디','맥주','평점'])
add3 = pd.DataFrame(data=[add3], columns=['아이디','맥주','평점'])
add4 = pd.DataFrame(data=[add4], columns=['아이디','맥주','평점'])
add5 = pd.DataFrame(data=[add5], columns=['아이디','맥주','평점'])

ratings = pd.concat([ratings, add1])
ratings = pd.concat([ratings, add2])
ratings = pd.concat([ratings, add3])
ratings = pd.concat([ratings, add4])
ratings = pd.concat([ratings, add5])

In [140]:
ratings

,아이디,맥주,평점
2,"snoworsummer(8,581)",Kloud Original Gravity,3.0
4,"GuideDogSaint(2,549)",Kloud Original Gravity,2.2
5,"deanso(13,380)",Kloud Original Gravity,2.3
7,"DrNosha(5,865)",Kloud Original Gravity,2.3
8,"mkel07(8,991)",Kloud Original Gravity,2.4
...,...,...,...
0,신구현,Asahi Super Dry,3.5
0,신구현,Hite Prime Max,3.0
0,신구현,Holsten Premium Bier,4.0
0,신구현,Erdinger Weissbier Dunkel,4.0


In [141]:
# 피벗 테이블로 변환 후 Nan값을 0으로 대체
ratings_matrix = ratings.pivot_table('평점', index='아이디', columns='맥주')
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix

맥주,Asahi Super Dry,Asahi Super Dry Black,Bavaria 8.6 (Original),Bavaria Pilsener / Premium Beer,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,Budweiser,Carlsberg Pilsner,Cass Fresh,...,Stephans Bräu Premium Lager,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Victoria Bitter,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
아이디,,,,,,,,,,,,,,,,,,,,,
007lund(92),0.0,0.0,0.0,0.0,2.6,0.0,0.0,2.0,1.8,0.0,...,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,4.1
"00cobraR(1,103)",0.0,0.0,0.0,0.0,2.3,0.0,0.0,1.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0
1010(309),2.4,0.0,0.0,0.0,2.0,0.0,0.0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12HundredBaud(262),2.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16erBlech(561),0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ziobarba2000(91),0.0,0.0,0.0,1.6,1.6,0.0,0.0,2.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4
zombywoof(483),0.0,0.0,0.0,1.5,1.8,0.0,0.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5
"zvikar(11,249)",1.2,2.5,0.0,0.0,2.1,0.0,0.0,1.4,1.7,1.9,...,2.2,0.0,0.0,1.7,0.0,1.7,0.0,1.4,0.0,2.3


In [142]:
tmpP = pd.DataFrame(data=P)
tmpP

,0,1,2,3,4,5,6,7,8,9
0,1.005130,-0.449008,0.244887,0.745418,-0.954999,0.172556,1.189754,0.064679,-0.077123,-0.269131
1,-0.105118,-0.568701,0.837593,0.918876,0.320704,0.079374,1.123356,0.080196,0.008840,-0.225152
2,0.629064,-0.580893,0.888949,1.120777,0.402671,-0.277592,0.707599,-0.193684,-0.047768,-0.440958
3,0.230888,-0.524909,0.648022,1.111746,0.034749,0.662277,1.168410,-0.105589,0.142978,-0.153025
4,0.271216,-0.236859,0.998508,0.734945,-0.182947,0.202776,1.078457,0.785397,-0.007934,-0.533649
...,...,...,...,...,...,...,...,...,...,...
2491,-0.091791,-0.964660,0.778555,0.262834,-0.152936,0.381640,1.029551,0.472997,0.255018,-0.274915
2492,0.265170,0.100464,0.101781,1.327524,-0.059951,0.102945,0.869606,0.422074,-0.228389,0.495701
2493,0.703176,0.134798,0.771313,0.805424,0.659334,0.380500,0.825771,0.335146,0.403025,-0.405732
2494,0.085155,0.064006,0.871946,0.877362,0.165633,0.229027,1.051082,0.381674,-0.023665,0.006310


In [143]:
ratings_matrix.shape

(2497, 65)

In [144]:
t_P = np.random.normal(scale=1./10, size=(1, 10))
t_P = pd.DataFrame(data=t_P)
t_P

,0,1,2,3,4,5,6,7,8,9
0,0.025284,-0.006686,-0.00519,-0.011691,-0.06235,0.10241,0.004989,-0.1319,0.286552,-0.1298


In [145]:
tmpP = pd.concat([tmpP, t_P])
tmpP

,0,1,2,3,4,5,6,7,8,9
0,1.005130,-0.449008,0.244887,0.745418,-0.954999,0.172556,1.189754,0.064679,-0.077123,-0.269131
1,-0.105118,-0.568701,0.837593,0.918876,0.320704,0.079374,1.123356,0.080196,0.008840,-0.225152
2,0.629064,-0.580893,0.888949,1.120777,0.402671,-0.277592,0.707599,-0.193684,-0.047768,-0.440958
3,0.230888,-0.524909,0.648022,1.111746,0.034749,0.662277,1.168410,-0.105589,0.142978,-0.153025
4,0.271216,-0.236859,0.998508,0.734945,-0.182947,0.202776,1.078457,0.785397,-0.007934,-0.533649
...,...,...,...,...,...,...,...,...,...,...
2492,0.265170,0.100464,0.101781,1.327524,-0.059951,0.102945,0.869606,0.422074,-0.228389,0.495701
2493,0.703176,0.134798,0.771313,0.805424,0.659334,0.380500,0.825771,0.335146,0.403025,-0.405732
2494,0.085155,0.064006,0.871946,0.877362,0.165633,0.229027,1.051082,0.381674,-0.023665,0.006310
2495,0.515097,-0.277580,0.607185,0.908831,-0.003898,0.119615,0.834483,0.508221,0.209145,-0.361770


In [148]:
def matrix_factorization(R,P,Q,K, steps=100, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape
    
    # P와 Q의 매트릭스 크기를 지정하고 정규 분포를 가진 랜덤한 값으로 입력
#     np.random.seed(1)
#     P = np.random.normal(scale=1./K, size=(num_users, K))
#     Q = np.random.normal(scale=1./K, size=(num_items, K))
    
    prev_rmse = 10000
    break_count = 0
    
    prev_rmse = 10000
    break_count = 0
    
    # R>0인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장.
    non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]
    
    # SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i,:], Q[j,:].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
            
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 5) == 0:
            print('### iteration step : ', step, " rmse: ", rmse)
        
    return P,Q

In [149]:
# 아예 학습된 P,Q를 넣어버리면 어떻게 되지?
res_P, res_Q = matrix_factorization(ratings_matrix.values, tmpP.values, Q, K=10, steps=10, learning_rate=0.01, r_lambda=0.02)

res_pred_matrix = np.dot(res_P, res_Q.T)

### iteration step :  0  rmse:  0.3058835299767472
### iteration step :  5  rmse:  0.30549776873936413


In [164]:
res_ratings_pred_matrix = pd.DataFrame(data=res_pred_matrix, index=ratings_matrix.index,
                                  columns=ratings_matrix.columns)
res_ratings_pred_matrix.tail(3)

맥주,Asahi Super Dry,Asahi Super Dry Black,Bavaria 8.6 (Original),Bavaria Pilsener / Premium Beer,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,Budweiser,Carlsberg Pilsner,Cass Fresh,...,Stephans Bräu Premium Lager,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Victoria Bitter,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
아이디,,,,,,,,,,,,,,,,,,,,,
"zvikar(11,249)",1.505119,2.791860,1.969153,2.205895,2.182856,2.493530,2.048657,1.729654,2.091911,1.841974,...,1.892206,2.122737,2.119415,1.954239,2.214614,1.970109,3.156741,1.578303,2.771749,2.237058
"zvsn(5,413)",2.019872,2.677801,2.209028,2.192304,1.984391,2.452078,2.104894,1.273616,1.970197,1.683072,...,1.784033,1.885968,2.277016,2.124695,1.822490,2.125226,2.776348,2.108130,2.773367,2.673608
신구현,3.048597,3.279326,3.154552,2.883757,2.967068,3.407623,2.832402,2.246988,3.027253,2.572867,...,2.856475,2.369398,3.502929,2.761002,2.501279,2.596942,3.389898,2.996939,3.711550,3.854108


In [170]:
res_ratings_pred_matrix.tail(1).to_excel('신구현.xlsx')

In [60]:
# 사용자가 안 먹어본 맥주를 추천하자.
def get_not_tried_beer(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 맥주 정보를 추출해 Series로 반환
    # 반환된 user_rating은 영화명(title)을 인덱스로 가지는 Series 객체
    user_rating = ratings_matrix.loc[userId, :]
    
    # user_rating이 0보다 크면 기존에 관란함 영화.
    # 대상 인덱스를 추출해 list 객체로 만듦
    tried = user_rating[user_rating>0].index.tolist()
    
    # 모든 맥주명을 list 객체로 만듦
    beer_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 tried에 해당하는 영화는 beer_list에서 제외
    not_tried = [beer for beer in beer_list if beer not in tried]
    
    return not_tried

In [61]:
# 예측 평점 DataFrame에서 사용자 id 인덱스와 not_tried로 들어온 맥주명 추출 후
# 가장 예측 평점이 높은 순으로 정렬
def recomm_beer_by_userid(pred_df, userId, not_tried, top_n):
    recomm_beer = pred_df.loc[userId, not_tried].sort_values(ascending=False)[:top_n]
    return recomm_beer

In [154]:
ratings_matrix

맥주,Asahi Super Dry,Asahi Super Dry Black,Bavaria 8.6 (Original),Bavaria Pilsener / Premium Beer,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,Budweiser,Carlsberg Pilsner,Cass Fresh,...,Stephans Bräu Premium Lager,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Victoria Bitter,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
아이디,,,,,,,,,,,,,,,,,,,,,
007lund(92),0.0,0.0,0.0,0.0,2.6,0.0,0.0,2.0,1.8,0.0,...,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,4.1
"00cobraR(1,103)",0.0,0.0,0.0,0.0,2.3,0.0,0.0,1.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0
1010(309),2.4,0.0,0.0,0.0,2.0,0.0,0.0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12HundredBaud(262),2.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16erBlech(561),0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ziobarba2000(91),0.0,0.0,0.0,1.6,1.6,0.0,0.0,2.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4
zombywoof(483),0.0,0.0,0.0,1.5,1.8,0.0,0.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5
"zvikar(11,249)",1.2,2.5,0.0,0.0,2.1,0.0,0.0,1.4,1.7,1.9,...,2.2,0.0,0.0,1.7,0.0,1.7,0.0,1.4,0.0,2.3


In [155]:
undrunken_list = get_not_tried_beer(ratings_matrix, '신구현')
undrunken_list

['Asahi Super Dry Black',
 'Bavaria 8.6 (Original)',
 'Bavaria Pilsener / Premium Beer',
 "Beck's",
 'Berliner Kindl Jubiläums Pilsener Premium',
 'Bintang Pilsener',
 'Budweiser',
 'Carlsberg Pilsner',
 'Cass Fresh',
 'Castlemaine XXXX Gold Lager',
 'Desperados',
 'Edelweiss Weissbier Snowfresh',
 'Egger Grapefruit Radler',
 'Egger Märzenbier',
 'Eichbaum Apostel Bräu',
 'Erdinger Weissbier',
 'Franziskaner Hefe-Weissbier / Weissbier Naturtrub',
 'Gambrinus Originál 10° (Desítka)',
 'Grolsch Premium Lager',
 'Guinness Draught',
 'Guinness Original 4.2% (Ireland/UK)',
 'Harbin Beer',
 'Heineken',
 'Heineken Dark Lager',
 'Hite D (Dry Finish)',
 'Hite Extra Cold',
 'Hoegaarden',
 'Hoegaarden Rosée',
 'Jeju Wit Ale',
 'Kirin Ichiban',
 'Kloud Fitz Super Clear',
 'Kloud Original Gravity',
 'Kozel Černý (Dark) 10°',
 'Krombacher Weizen',
 'Kronenbourg 1664 Blanc',
 'Lapin Kulta IV A',
 'Leffe Brune / Bruin / Brown',
 'Löwenbräu Original (4.2%)',
 'Miller Genuine Draft (MGD)',
 'OB Premier 

In [156]:
ratings[ratings['아이디']=='신구현']

,아이디,맥주,평점
0,신구현,Asahi Super Dry,3.5
0,신구현,Hite Prime Max,3.0
0,신구현,Holsten Premium Bier,4.0
0,신구현,Erdinger Weissbier Dunkel,4.0
0,신구현,Red Rock,4.0


In [157]:
recomm_beers = recomm_beer_by_userid(ratings_pred_matrix, '신구현',
                                     undrunken_list, top_n=10)
recomm_beers

맥주
Paulaner Hefe-Weissbier                              4.518858
Franziskaner Hefe-Weissbier / Weissbier Naturtrub    4.467182
Leffe Brune / Bruin / Brown                          4.393589
Erdinger Weissbier                                   4.275437
Pilsner Urquell                                      4.140452
Edelweiss Weissbier Snowfresh                        4.083448
Hoegaarden                                           4.076168
Guinness Original 4.2% (Ireland/UK)                  4.055447
Krombacher Weizen                                    3.987622
Guinness Draught                                     3.949651
Name: 신구현, dtype: float64

# MF를 빠르게 실행할 수 있는 방법은 없을까?

In [ ]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape
    
    # P와 Q의 매트릭스 크기를 지정하고 정규 분포를 가진 랜덤한 값으로 입력
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))
    
    prev_rmse = 10000
    break_count = 0
    
    prev_rmse = 10000
    break_count = 0
    
    # R>0인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장.
    non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]
    
    # SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i,:], Q[j,:].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
            
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0:
            print('### iteration step : ', step, " rmse: ", rmse)
        
    return P,Q

In [74]:
num_users, num_items = ratings_matrix.shape

In [75]:
num_users, num_items

(2496, 65)

# Surpise로 사용하면 더 빠를까? 비교하기

In [172]:
data=pd.read_csv('맥주_10.csv', encoding='utf-8', index_col=0)

In [173]:
ratings=data.copy()

In [174]:
ratings

,아이디,맥주,평점
2,"snoworsummer(8,581)",Kloud Original Gravity,3.0
4,"GuideDogSaint(2,549)",Kloud Original Gravity,2.2
5,"deanso(13,380)",Kloud Original Gravity,2.3
7,"DrNosha(5,865)",Kloud Original Gravity,2.3
8,"mkel07(8,991)",Kloud Original Gravity,2.4
...,...,...,...
2446,beerman991979(136),Warsteiner Premium Verum,3.2
2448,"Aubrey(3,516)",Warsteiner Premium Verum,2.9
2449,Sammys(427),Warsteiner Premium Verum,2.9
2450,"Bov(14,984)",Warsteiner Premium Verum,2.2
